# Backpropagation

![](img/bp7.png)
-----------------------
### Chain Rule is Handy 
![](img/chainrule.png)
----------------------------------
![](img/bp10.png)

### Sigmoid Dervative
![](img/sig.png)

### EX-1 Working through a simple example
![](img/ex1.png)

In [ ]:
import numpy as np
import pandas as pd

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

In [ ]:
x = np.array([0.1, 0.3])
target = 1 # Binary classification
weights_input_hidden = np.array([0.4, -0.2])
weights_hidden_output = np.array([0.1])

##### Forward pass

In [ ]:
hidden_layer_input = np.dot(x, weights_input_hidden)
hidden_layer_output = sigmoid(hidden_layer_input)

output_layer_in = hidden_layer_output * weights_hidden_output
output = sigmoid(output_layer_in)

In [ ]:
hidden_layer_input

In [ ]:
0.1 * 0.4 - 0.2 * 0.3

In [ ]:
hidden_layer_output

In [ ]:
1 / (1 + np.exp(-(-0.01999999999999999)))

![](img/f1.PNG)

In [ ]:
output_layer_in

In [ ]:
0.1 * hidden_layer_output

In [ ]:
output

In [ ]:
1 / (1 + np.exp(-output_layer_in))

##### Backward Pass

![](img/f2.PNG)
![](img/f3.PNG)

In [ ]:
## Calculate output error
error = target - output

In [ ]:
error

In [ ]:
# Calculate error term for output layer
output_error_term = error * output * (1 - output)
output_error_term

In [ ]:
learnrate = 0.5

In [ ]:
# Calculate change in weights for hidden layer to output layer
delta_w_h_o = learnrate * output_error_term * hidden_layer_output

### Error term for Hidden unit
![](img/f4.png)

In [ ]:
# Calculate error term for hidden layer
hidden_error_term = np.dot(output_error_term, weights_hidden_output) * \
                    hidden_layer_output * (1 - hidden_layer_output)

In [ ]:
# Calculate change in weights for input layer to hidden layer
delta_w_i_h = learnrate * hidden_error_term * x[:, None]

In [ ]:
print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_h)

### EX-2 

In [ ]:
x = np.array([0.5, 0.1, -0.2])
target = 0.6
learnrate = 0.5


weights_input_hidden = np.array([[0.5, -0.6],
                                 [0.1, -0.2],
                                 [0.1, 0.7]])

weights_hidden_output = np.array([0.1, -0.3])

## Forward pass
hidden_layer_input = np.dot(x, weights_input_hidden)
hidden_layer_output = sigmoid(hidden_layer_input)

output_layer_in = np.dot(hidden_layer_output, weights_hidden_output)
output = sigmoid(output_layer_in)

## Backwards pass
## Calculate output error
error = target - output

# Calculate error term for output layer
output_error_term = error * output * (1 - output)

# Calculate error term for hidden layer
hidden_error_term = np.dot(output_error_term, weights_hidden_output) * \
                    hidden_layer_output * (1 - hidden_layer_output)

# Calculate change in weights for hidden layer to output layer
delta_w_h_o = learnrate * output_error_term * hidden_layer_output

# Calculate change in weights for input layer to hidden layer
delta_w_i_h = learnrate * hidden_error_term * x[:, None]

print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_h)

## Ex-3 Implementing Gradient Descent with Students Data set

In [ ]:
admissions = pd.read_csv('data/binary.csv')
# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)
data

In [ ]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std

In [ ]:
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [ ]:
# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None


# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)


# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

In [ ]:
# Initial accuracy on test data using the initialized weights
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

In [ ]:
for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Activation of the output unit
        #   Notice we multiply the inputs and the weights here 
        #   rather than storing h as a separate variable 
        output = sigmoid(np.dot(x, weights))

        # The error, the target minus the network output
        error = y - output

        # The error term
        #   Notice we calulate f'(h) here instead of defining a separate
        #   sigmoid_prime function. This just makes it faster because we
        #   can re-use the result of the sigmoid function stored in
        #   the output variable
        error_term = error * output * (1 - output)

        # The gradient descent step, the error times the gradient times the inputs
        del_w += error_term * x

    # Update the weights here. The learning rate times the 
    # change in weights, divided by the number of records to average
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

In [ ]:
print ("Final Weights:", weights)

In [ ]:
# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))